In [4]:
from neo4j import GraphDatabase
from tabulate import tabulate
URI = "neo4j://localhost:7687" 
USERNAME = "neo4j"
PASSWORD = "password"

In [5]:
def execute_query(query, params=None):
    """Execute a Cypher query and return results."""
    with GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD)) as driver:
        with driver.session() as session:
            result = session.run(query, params or {})
            return [record.data() for record in result]

In [6]:
def print_table(data):
    if data:
        print(tabulate(data, headers="keys", tablefmt="grid"))

In [17]:
# Get 10 hospitals and the list of specializations they provide
query1 = """MATCH (h:Hospital)<-[:WORKS_AT]-(p:Provider)-[:SPECIALIZES_IN]->(t:Taxonomy)
RETURN h.FacilityName AS Hospital,
COLLECT(DISTINCT t.Description) AS Specializations,
COLLECT(DISTINCT p.FirstName) AS Providers
LIMIT 10;"""

result = execute_query(query1)
print_table(result)


+-----------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [18]:
# Get the hospital and its count with the maximum number of specialties offered
query2 = """MATCH (h:Hospital)<-[:WORKS_AT]-(p:Provider)-[:SPECIALIZES_IN]->(t:Taxonomy)
RETURN h.FacilityName AS Hospital, COUNT(DISTINCT t) AS SpecializationCount
ORDER BY SpecializationCount DESC
LIMIT 1;"""

result = execute_query(query2)
print_table(result)


+-----------------------------------+-----------------------+
| Hospital                          |   SpecializationCount |
+===================================+=======================+
| RONALD REAGAN UCLA MEDICAL CENTER |                    78 |
+-----------------------------------+-----------------------+


In [19]:
# Given a city and specialty, get the facility name and the number of providers that specialize in it
query3 = """MATCH (c:City {name: 'SAN FRANCISCO'})<-[:IN_CITY]-(h:Hospital)
MATCH (h)<-[:WORKS_AT]-(p:Provider)-[:SPECIALIZES_IN]->(t:Taxonomy)
WHERE t.Description = 'Physician Assistants & Advanced Practice Nursing Providers/Nurse Practitioner, Acute Care'
RETURN h.FacilityName, COUNT(p) AS provider_count
ORDER BY provider_count DESC;"""

result = execute_query(query3)
print_table(result)

+-------------------------------------------------------------+------------------+
| h.FacilityName                                              |   provider_count |
+=============================================================+==================+
| ZUCKERBERG SAN FRANCISCO GENERAL HOSPITAL AND TRAUMA CENTER |                1 |
+-------------------------------------------------------------+------------------+


In [20]:
# Connection between 2 hospitals
query4 = """MATCH p=shortestPath((h1:Hospital {FacilityName: 'HOLT MEDICAL CENTER'})-[:WORKS_AT|SPECIALIZES_IN*]-(h2:Hospital {FacilityName: 'ANZA COMMUNITY HEALTH'}))
RETURN p;"""

result = execute_query(query4)
print_table(result)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [21]:
# Gets the path connecting a hospital to a specialty; can find the closest hospital based on that specialty in relationships

query5 = """MATCH p=shortestPath((h1:Hospital {FacilityName: 'HOLT MEDICAL CENTER'})-[:WORKS_AT|SPECIALIZES_IN*]-(t:Taxonomy {Description: 'Physician Assistants & Advanced Practice Nursing Providers/Nurse Practitioner, Acute Care'}))
RETURN p;"""

result = execute_query(query5)
print_table(result)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
# Hospital with the most number of doctors
query6 = """MATCH (h:Hospital)<-[:WORKS_AT]-(p:Provider)
WITH h, COUNT(p) AS providerCount
RETURN h.FacilityName AS hospitalName, providerCount
ORDER BY providerCount DESC LIMIT 20;"""
result = execute_query(query6)
print_table(result)

+------------------------------------------------------+-----------------+
| hospitalName                                         |   providerCount |
+======================================================+=================+
| MAXIM HEALTHCARE SERVICES HOLDINGS INC               |            4039 |
+------------------------------------------------------+-----------------+
| UCSF MEDICAL CTR                                     |            3153 |
+------------------------------------------------------+-----------------+
| BEHAVIORAL HEALTH WORKS                              |            2462 |
+------------------------------------------------------+-----------------+
| LOS ANGELES MEDICAL CENTER-LAL                       |            2037 |
+------------------------------------------------------+-----------------+
| OC ANAHEIM MEDICAL CENTER-OCAL                       |            1645 |
+------------------------------------------------------+-----------------+
| RONALD REAGAN UCLA MEDI

In [13]:
#  The most connected doctor -> works in multiple locations
query7 = """MATCH (p:Provider)-[:WORKS_AT]->(h:Hospital)
WITH p.FirstName + ' ' + p.LastName AS providerName,
COLLECT(DISTINCT h) AS hospitals,
COUNT(DISTINCT h) AS hospitalCount
RETURN providerName, hospitalCount,
[hospital IN hospitals | hospital.FacilityName] AS hospitalNames
ORDER BY hospitalCount DESC LIMIT 20;"""
result = execute_query(query7)
print_table(result)

+-----------------------+-----------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| providerName          |   hospitalCount | hospitalNames                                                                                                                                                                                                                       |
+=======================+=================+=====================================================================================================================================================================================================================================+
| FIDEL SANTACRUZ       |               6 | ['LOS ANGELES COMMUNITY HOSPITAL', 'ST. FRANCIS MEDICAL CENTER', 'MONTEREY PARK HOSPITAL', 'ALHAMBRA HOSPITAL MEDICAL CENTER', "EAST L

In [14]:
# Hospital with the most insurances covered; or most covered insurance
query8 = """MATCH (h:Hospital)<-[:WORKS_AT]-(p:Provider)-[:PART_OF]->(sn:SubNetwork)-[:UNDER]->(mcp:ManagedCarePlan)
WITH h, COLLECT(DISTINCT mcp) AS managedCarePlans
WITH h, managedCarePlans, SIZE(managedCarePlans) AS planCount
RETURN h.FacilityName AS hospitalName, planCount, [mcp IN managedCarePlans | mcp.name] AS managedCarePlanNames
ORDER BY planCount DESC LIMIT 20;"""
result = execute_query(query8)
print_table(result)

+---------------------------------------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| hospitalName                                |   planCount | managedCarePlanNames                                                                                                                                                                                                                                           |
+=============================================+=============+================================================================================================================================================================================================================================================================+
| TRUECARE                                 

In [16]:
query9 = """MATCH (p1:Provider)-[:SPECIALIZES_IN]->(t:Taxonomy)<-[:SPECIALIZES_IN]-(p2:Provider)
WHERE p1.FirstName + " " + p1.LastName < p2.FirstName + " " + p2.LastName
WITH p1.FirstName + " " + p1.LastName AS Doctor1,
     p2.FirstName + " " + p2.LastName AS Doctor2,
     t.Description AS SharedSpecialty
WITH DISTINCT Doctor1, Doctor2, SharedSpecialty
LIMIT 100
RETURN Doctor1, Doctor2, SharedSpecialty
ORDER BY Doctor1, Doctor2 LIMIT 20;"""
result = execute_query(query9)
print_table(result)


+------------------+---------------------------+------------------------------------------------------+
| Doctor1          | Doctor2                   | SharedSpecialty                                      |
+==================+===========================+======================================================+
| Azra Zarrinpar   | JONATHAN LYNNE            | Allopathic & Osteopathic Physicians/General Practice |
+------------------+---------------------------+------------------------------------------------------+
| Azra Zarrinpar   | PETIA KREMEN              | Allopathic & Osteopathic Physicians/General Practice |
+------------------+---------------------------+------------------------------------------------------+
| Azra Zarrinpar   | SHAHRZAD SODAGAR-MARVASTI | Allopathic & Osteopathic Physicians/General Practice |
+------------------+---------------------------+------------------------------------------------------+
| Azra Zarrinpar   | XIAOGUANG WANG            | Allopathic & Os